# Wirecard Data Challenge

## Case 1: - Modelagem dimensional

Um dos desafios é definir o modelo dimensional para a tarifação das contas na empresa. têm-se que efetuar consultas SQL em uma série de tabelas e considerando diversas regras para se conseguir obter a informação desejada. O primeiro desafio será definir um modelo dimensional que otimize o acesso a este tipo de informação.

Além das informações sobre as taxas, através do modelo dimensional proposto deve-se ser capaz de acessar algumas informações detalhadas das contas (login, nome, etc.).

#### Regras de Negócio

Obrigatoriamente, deve-se aplicar duas taxas em todo e qualquer pagamento: uma taxa fixa (definida em reais); e uma taxa percentual, que deve ser aplicada com base no valor total do pagamento. Os valores dessas taxas podem variar de acordo com o modelo de precificação.

Além dessas duas taxas, caso o cliente queira antecipar o valor que ele tem para receber, deve-se aplicar uma terceira taxa, a taxa de antecipação. Para esta taxa, as contas Moip são tarifadas de acordo com o modelo de precificação definido em contrato.

Atualmente temos três modelos de precificação:

Modelo 1: Taxa percentual: aplicada de acordo com o tipo de pagamento.

Taxa de antecipação: aplicar uma taxa fixa de 1,99% a.m. (juros compostos)

Modelo 2: Taxa percentual: aplicada de acordo com o tipo de pagamento e agrupamento de quantidade de parcelas.

Taxa de antecipação: aplicar uma taxa percentual de acordo com o meio de pagamento. Portanto, cada meio de pagamento terá sua taxa específica. Caso um meio de pagamento não tenha uma taxa atrelada, devemos considerar a mesma taxa de antecipação utilizada pelo meio de pagamento Cartão de crédito.

Modelo 3: Taxa percentual: aplicada de acordo com o tipo de pagamento e quantidade de parcelas.

Taxa de antecipação: aplicar uma taxa fixa de 2,89% a.m. (juros simples) sobre a quantidade de dias que serão antecipados.

Além dos 3 modelos descritos acima, embora cada uma das contas Moip possua suas respectivas taxas, caso ela transacione (receba pagamentos) através de um channel, as taxas que serão aplicadas àquela conta serão as taxas do channel, desde que a conta não possua uma taxa negociada.

Portanto, a precedência das taxas ocorre da seguinte forma (maior para menor prioridade): taxa da conta se houver negociação, taxa do channel e taxa da conta.

Um channel também possui uma conta Moip. Portanto, as taxas do channel são definidas da mesma forma que as taxas de uma conta Moip comum. Ou seja, seguindo as mesmas regras descritas acima, com exceção de que um channel não pode transacionar por outro channel.

## Modelo Dimensional

Dadas as características desejadas, será implementado banco de dados com arquitetura estrela ( ou estrela de Kimball), esse é o modelo dimensional aconselhado para essas situações no livro The Data Warehouse Toolkit, 3rd Edition.

O primeiro passo é avaliar qual processo será modelado em fatos. Na descrição do problema, logo no primeiro parágrafo pode-se identificar que o objetivo é modelar tarifação das contas na empresa.

O segundo passo seria definir qual a granularidade desses dados, mas como não vemos dimensão de tempo no banco de dados operacional, essa etapa não será feita.

Para identificar o que será fato e o que será dimensão, é melhor analisar o banco de dados operacional.

#### Análise do banco de dados operacional

Os dados de cada base de dados estão nos seguintes arquivos com formato CSV

In [1]:
from os import listdir
import os
filenames = listdir(os.getcwd())
filenames_list = [ filename for filename in filenames if filename.endswith( ".csv" ) ]
filenames_list

['account.csv',
 'accounts_channels.csv',
 'account_fixed_table_fee.csv',
 'account_payment_fee.csv',
 'channel.csv',
 'channel_fixed_table_fee.csv',
 'fee_account.csv',
 'fixed_table_fee.csv',
 'member.csv',
 'payment_form.csv',
 'tax_applied_to_account.csv']

A partir da análise desses dados, pode-se construir o seguinte diagrama entidade-relação

<img src="wirecard_opdb.jpeg">

nota: pode ser que algumas dessas relações de cardinalidade estejam equivocadas

#### Fato: tarifação das contas na empresa

Como pode-se notar, existem 4 tabelas com informação sobre o valor de tarifação. Precisamos organizar os dados nessas tabelas de forma que essas informações sejam acessadas de forma mais conveniente por analistas e executivos a fim de se facilitar a tomada de decisão e planejamento.

Kimball recomenda em seu livro, preservar todos as informações relacionadas aos fatos possíveis, porque as necessidades do time de BI e analisatas pode mudar com o tempo, e dados irrelevantes hoje podem ser necessários para alguma análise no futuro, por isso, todas as informações nas tabelas serão preservadas na construção do modelo dimensional.

Outra preocupação que deve ser levada em conta, é utilizar nomes que façam sentido para todos os usuários, na hora de fazer o projeto de um Data Werehouse, é desencorajado o uso de abreaviações que não são convenção de todos ou que possam dificultar o entedimento de usuários no futuro.

Existem 3 tarifas principais sendo as duas primeiras obrigatórias,

1 - Tarifa Fixa

2 - Tarifa percentual, que deve ser aplicada com base no valor total do pagamento

3 - Tarifa de antecipação, caso o cliente queira antecipar o valor que ele tem para receber


A abordagem utilizada, será a de calcular o valor dessas tres tarifas utilizando os dados nas outras tabelas para deixar esse valores facilmente acessiveis na tabelas fato (a tabela central)

Com isso, uma possível implementação do esquema em estrela é como o abaixo:

<img src="wirecard_stardb.jpeg">


## Case 2: - ETL

O objetivo das próximas sessões será colocar os dados dos banco de dados operacional no esquema estrela proposto. O processo de ETL será feito com scripts Python, essa escolha é motiva por vários motivos, entre elas, a grande disponibilidade de bibliotecas úteis no processo e o fato de várias ferramentas de ETL hoje rodarem scripts Python, uma que se destaca nos últimos anos é o <b>Apache Airflow</b>. Com o Airflow seria possível automatizar todo o processo a partir dos scripts que serão desenvolvidos nas próximas sessões.

Serão usados Dataframes Pandas para manipular os dados, essa escolha é motivada por 2 motiovos:

1 - Devido a forma de indexação do Pandas, ele é muito rápido

2 - Os Dataframes Pandas possuem muitos métodos que facilitam o processo ETL, em especial, o método merge do pandas realiza função análoga aos comandos join da linguagem SQL, que serão muito úteis a seguir

Existem 3 modelos de precificação, por isso 3  scripts de transformações diferentes precisam ser escritos separadamente.

O primeiro passo será carregar todos os dados dos arquivos CSV em Dataframes


In [2]:
import pandas as pd

# iterando sobre a lista de nome de arquivos CSV
# criar Dataframe com o mesmo nome do arquivo
# Os arquivos tem separação de decimais com vírgula, por isso é preciso passar parâmetros identificando decimal

for filename in filenames_list:
    globals()[filename.strip('.csv')] = pd.read_csv(filename, decimal=",", sep=';')

# TODO: Por algum motivo esses 2 arquivos não estavam carregando dentro do loop acima
accounts_channels = pd.read_csv('accounts_channels.csv', decimal=",", sep=';')
channel_fixed_table_fee = pd.read_csv('channel_fixed_table_fee.csv', decimal=",", sep=';')

# como exemplo da estrutura dos dataframes
account.head()

,id,status,type,fee_type,has_bonus_withdraw,v2_integration_level,negotiated_tax,is_transparent
0,1005890,1,3,2,1,3,1,0
1,1005925,1,3,3,0,3,1,0
2,1122838,1,3,3,0,3,1,0
3,1122850,1,3,3,0,3,1,0
4,1122856,1,3,2,0,3,1,0


#### Modelo 1

Taxa percentual: aplicada de acordo com o tipo de pagamento. Taxa de antecipação: aplicar uma taxa fixa de 1,99% a.m. (juros compostos)

Após carregar os dados e fazer joins nós dados de account, member, fixed_table_fee e tax_applied_to_account calcularemos payment_fixed_tax, payment_percentual_tax e payment_antecipation_tax

In [3]:
# merge de account e member para obter informações completas sobre as contas
accountdf = pd.merge(account, member, how='inner', left_on='id', right_on='id')

In [17]:
# merge em payment_form_id para ter acesso aos valores das taxas tabeladas e das contas
# fixed_table_fee e tax_applied_to_account contém as informações de taxacao de contas com fee_type 1

#Para facilitar o acesso as informações, essas 2 principais tabelas serão processadas juntas
model1df = pd.merge(fixed_table_fee, tax_applied_to_account, how='inner',
         left_on='payment_form_id', right_on='payment_form_id')

# merge de informações das contas e pagamentos por conveniencia
model1df = pd.merge(model1df, accountdf, how='inner',
         left_on='account_id', right_on='id')

# Caso aja alguma conta fee_type diferente de 1 no dataframe, ela será excluida nesse processamento
model1df = model1df[model1df['fee_type'] == 1]

# Precendencia - Para a taxa fixa

# 1 - Se houver taxa negociada na conta, essa taxa será aplicada
# 2 - Se não, se a conta pertencer a um channel, a taxa do channel será aplicada
# 3 - se não, a taxa tabelada será aplicada

# Lista vazia para ser preenchida durante o loop com o valor da taxa fixa correta
payment_fixed_tax = []

# Loop por todas as linhas do dataframe para calcular a taxa fixa
for i in range(len(model1df)):
    
    # precedencia 1
    
    # Se houver taxa negociada
    if model1df['negotiated_tax'][i] == 1:
        # será aplicada taxa na conta
        payment_fixed_tax.append(model1df['fixed'][i])
      
    # precedencia 2
    # o código ficou um pouco grande, porque vamos procurar o valor da taxa no Dataframe channel_fixed_table_fee
    elif model1df['account_id'][i] in accounts_channels['account_id'].values:
        
        # Em resumo, todo esse bloco de código irá buscar o valor de fixed_tax_value na tabela channel_fixed_table_fee
        payment_fixed_tax.append(channel_fixed_table_fee[(channel_fixed_table_fee['channel_id']==int(accounts_channels[accounts_channels['account_id']== model1df['account_id'][i]]['channel_id'])) & 
                                (channel_fixed_table_fee['payment_form_id'] == model1df['payment_form_id'][i]) &
                                (channel_fixed_table_fee['parcel_number'] == model1df['parcel_number'][i])]['fixed_tax_value'][0])
    # precedencia 3
    
    else:
        
        # Taxa da tabela é aplicada
        payment_fixed_tax.append(model1df['fixed_tax_value'][i])

# Novo campo com os valores calculados é adicionado ao dataframe
model1df['payment_fixed_tax'] = payment_fixed_tax

# Procedimento análogo ao feito com a taxa fixa será feito para a taxa percentual

payment_percentual_tax = []

for i in range(len(model1df)):
    
    if model1df['negotiated_tax'][i] == 1:
        payment_percentual_tax.append(model1df['percentual'][i])
        
    elif model1df['account_id'][i] in accounts_channels['account_id'].values:
        payment_percentual_tax.append(channel_fixed_table_fee[(channel_fixed_table_fee['channel_id']==int(accounts_channels[accounts_channels['account_id']== model1df['account_id'][i]]['channel_id'])) & 
                                (channel_fixed_table_fee['payment_form_id'] == model1df['payment_form_id'][i]) &
                                (channel_fixed_table_fee['parcel_number'] == model1df['parcel_number'][i])]['fixed_tax_percentual'][0])
    else:
        payment_percentual_tax.append(model1df['fixed_tax_percentual'][i])
        
model1df['payment_percentual_tax'] = payment_percentual_tax

# Cálculo da taxa de antecipação com juros compostos com base na quantidade de parcelas
model1df['payment_antecipation_tax'] = [round(((1 + 0.0199)**n_parcelas - 1)*100, 3) for n_parcelas in model1df['parcel_number']]

# Agora apenas as colunas do fato serão mantidas no dataframe
model1df = model1df[['account_id', 'name', 'login', 'last_name', 'type', 'fee_type', 'negotiated_tax',
         'payment_form_id', 'payment_fixed_tax', 'payment_percentual_tax', 'payment_antecipation_tax', 'parcel_number']]

#### Modelo 2

Taxa percentual: aplicada de acordo com o tipo de pagamento e agrupamento de quantidade de parcelas.

Taxa de antecipação: aplicar uma taxa percentual de acordo com o meio de pagamento. Portanto, cada meio de pagamento terá sua taxa específica. Caso um meio de pagamento não tenha uma taxa atrelada, devemos considerar a mesma taxa de antecipação utilizada pelo meio de pagamento Cartão de crédito.

In [5]:
# account_payment_fee possui as informações de taxação para fee_type 2
# entretando essa tabela apenas não é suficiente para calcular todas as taxas
# o número de parcelas pode ser encontrado com um join com a tabela channel_fixed_table_fee

model2df = pd.merge(account_payment_fee, channel_fixed_table_fee, how='left',
         left_on='payment_form_id', right_on='payment_form_id')

# Join com o dataframe accountdf para obter também informações sobre as contas dos usuários
model2df = pd.merge(model2df, accountdf, how='inner',
         left_on='account_id', right_on='id')

model2df = model2df[model2df['fee_type'] == 2]

payment_fixed_tax = []

for i in range(len(model2df)):
    
    # precedencia 1
    
    # Se houver taxa negociada e a conta tiver taxa negociada na tabela tax_applied_to_account
    if model2df['negotiated_tax'][i] == 1 and model2df['account_id'][i] in tax_applied_to_account['account_id'].values:
        # Será aplicado o valor da conta
        payment_fixed_tax.append(tax_applied_to_account[tax_applied_to_account['account_id']==model2df['account_id'][i]]['fixed'][i])
      
    # precedencia 2
    # Se a conta está em algum channel, será aplicada a taxa do channel
    elif model2df['account_id'][i] in accounts_channels['account_id'].values:
        payment_fixed_tax.append(model2df['fixed_tax_value'][i])
    # precedencia 3
    else:
        # é aplicada taxa tabelada
        payment_fixed_tax.append(model2df['fixed'][i])

# Novos valores são adicionados ao dataframe
model2df['payment_fixed_tax'] = payment_fixed_tax

# O mesmo será feito para a taxa percentual
payment_percentual_tax = []

for i in range(len(model2df)):
    
    # precedencia 1
    if model2df['negotiated_tax'][i] == 1 and model2df['account_id'][i] in tax_applied_to_account['account_id']:
        payment_percentual_tax.append(tax_applied_to_account[tax_applied_to_account['account_id']==model2df['account_id'][i]]['percentual'][i])
      
    # precedencia 2
    elif model2df['account_id'][i] in accounts_channels['account_id'].values:
        payment_percentual_tax.append(model2df['fixed_tax_percentual'][i])
        
    # precedencia 3
    # A taxa varia de acordo com a quantidade de parcelas
    # Por isso precisamos verificar qual a quantidade de parcelas
    # Para atribuir a taxa correspondente
    else:
        if model2df['parcel_number'][i] == 1:
            payment_percentual_tax.append(model2df['one_parcel'][i])
            
        elif model2df['parcel_number'][i] >= 2 and model2df['parcel_number'][i] <= 6:
            payment_percentual_tax.append(model2df['between_two_and_six_parcels'][i])
            
        else:
            payment_percentual_tax.append(model2df['more_than_seven_parcels'][i])
        
model2df['payment_percentual_tax'] = payment_percentual_tax

# Taxa de antecipação será sempre a taxa tabelada em account_payment_fee
model2df['payment_antecipation_tax'] = model2df['antecipation_percentage']

# Nesse dataframe em particular, existem muitos valores duplicados, por isso eles serão excluidos
model2df = model2df.drop_duplicates(['id_x', 'payment_form_id'])

model2df = model2df[['account_id', 'name', 'login', 'last_name', 'type', 'fee_type', 'negotiated_tax',
         'payment_form_id', 'payment_fixed_tax', 'payment_percentual_tax', 'payment_antecipation_tax', 'parcel_number']]

#### Modelo 3

Taxa percentual: aplicada de acordo com o tipo de pagamento e quantidade de parcelas.

Taxa de antecipação: aplicar uma taxa fixa de 2,89% a.m. (juros simples) sobre a quantidade de dias que serão antecipados.

In [14]:
# Aqui será feito um processamento muito parecido ao feito para o modelo 1
# por isso poucos comentários serão feitos

model3df = pd.merge(account_fixed_table_fee, tax_applied_to_account, how='inner',
         left_on='payment_form_id', right_on='payment_form_id')

model3df['account_id'] = model3df['account_id_x']

# merge de informações das contas e pagamentos por conveniencia
model3df = pd.merge(model3df, accountdf, how='inner',
         left_on='account_id', right_on='id')

model3df = model3df[model3df['fee_type'] == 3]

# Precendencia - Para a taxa fixa

# 1 - Se houver taxa negociada na conta, essa taxa será aplicada
# 2 - Se não, se a conta pertencer a um channel, a taxa do channel será aplicada
# 3 - se não, a taxa tabelada será aplicada

payment_fixed_tax = []

for i in range(len(model3df)):
    
    # precedencia 1
    if model3df['negotiated_tax'][i] == 1:
        payment_fixed_tax.append(model3df['fixed'][i])
      
    # precedencia 2
    
    # verificar se a conta está atrelada a algum canal
    # o código ficou um pouco grande, porque vamos procurar o valor da taxa no Dataframe channel_fixed_table_fee
    elif model3df['account_id'][i] in accounts_channels['account_id'].values:
        payment_fixed_tax.append(channel_fixed_table_fee[(channel_fixed_table_fee['channel_id']==int(accounts_channels[accounts_channels['account_id']== model3df['account_id'][i]]['channel_id'])) & 
                                (channel_fixed_table_fee['payment_form_id'] == model3df['payment_form_id'][i]) &
                                (channel_fixed_table_fee['parcel_number'] == model3df['parcel_number'][i])]['fixed_tax_value'][0])
    # precedencia 3
    else:
        payment_fixed_tax.append(model3df['fixed_tax_value'][i])
        
model3df['payment_fixed_tax'] = payment_fixed_tax

# Procedimento análogo ao feito com a taxa fixa será feito para a taxa percentual

payment_percentual_tax = []

for i in range(len(model3df)):
    
    if model3df['negotiated_tax'][i] == 1:
        payment_percentual_tax.append(model3df['percentual'][i])
        
    elif model3df['account_id'][i] in accounts_channels['account_id'].values:
        payment_percentual_tax.append(channel_fixed_table_fee[(channel_fixed_table_fee['channel_id']==int(accounts_channels[accounts_channels['account_id']== model3df['account_id'][i]]['channel_id'])) & 
                                (channel_fixed_table_fee['payment_form_id'] == model3df['payment_form_id'][i]) &
                                (channel_fixed_table_fee['parcel_number'] == model3df['parcel_number'][i])]['fixed_tax_percentual'][0])
    else:
        payment_percentual_tax.append(model3df['fixed_tax_percentual'][i])
        
model3df['payment_percentual_tax'] = payment_percentual_tax

# Cálculo da taxa de antecipação com juros simples com base na quantidade de parcelas
model3df['payment_antecipation_tax'] = [round((2.89)*n_parcelas, 3) for n_parcelas in model3df['parcel_number']]

# Agora apenas as colunas do fato serão mantidas no dataframe
model3df = model3df[['account_id', 'name', 'login', 'last_name', 'type', 'fee_type', 'negotiated_tax',
         'payment_form_id', 'payment_fixed_tax', 'payment_percentual_tax', 'payment_antecipation_tax', 'parcel_number']]

In [18]:
# Agora, para obter a tabela fee_account completa, precisamos apenas juntar as informações
# dos 3 modelos de precificação
fee_account = pd.concat([model1df, model2df, model3df])
# Todos os valores de fee_account['account_id'] estão como 1 zero depois da virgula
# por isso terão conversão forçada para int
fee_account['account_id'] = fee_account['account_id'].apply(lambda x: int(x))

In [21]:
# Arquivo csv é escrito nos disco
fee_account.to_csv('fee_account.csv', encoding='utf-8', index=False, decimal=",", sep=';')

Como as tabelas account, payment_form, accounts_channel, channel permanescem inalteradas, eles não precisam ser processadas durante o ETL.

Com isso, é possível fazer consultas simples na tabela fee_account, que é a tabela fato, e caso alguma informação adicional seja necessária, no máximo uma join simples precisará ser feita para acessar as informalções nas tabelas dimensionais.